# Práctica tres

Grupo 14:
* Joaquín Ibáñez Penalva
* Aurora Zuoris

Para la realización de esta práctica  se usará la librería de numpy, pandas, matplotlib y sklearn.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn.linear_model as lm
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('precio_casas.csv', sep=';')
df.head()

# Ejercicio 1

In [ ]:
data = df.to_numpy()
x = data[:, :-1] # todas las filas, todas las columnas menos la última
y = data[:, -1] # todas las filas, la última columna
#crear vectores de la matriz de datos
#print(df.columns)
fig, ax = plt.subplots(2, 4, figsize=(32, 16))
idx_to_r2 = []
for i, (header, vec) in enumerate(zip(df.columns, x.T)):
    col = vec.reshape(-1, 1)
    #print(header, col)
    reg = lm.LinearRegression().fit(col, y)
    r2 = r2_score(y, reg.predict(col))
    idx_to_r2.append((i, r2))
    print(f"{header}: {r2}")
    ax[i // 4, i % 4].scatter(vec, y, color='blue')
    ax[i // 4, i % 4].plot(col, col * reg.coef_[0] + reg.intercept_, color='red')
    ax[i // 4, i % 4].set_title(f"{header}: {r2:.2f}")
    ax[i // 4, i % 4].set_ylim(0, 1.05 * y.max())


# Ejercicio 2

Primero se dividen los datos en un conjunto de entrenamiento y otro de test.

In [ ]:
data = df.to_numpy()

X, y = data[:, :-1], data[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print(X_train.shape, y_train.shape, )

In [ ]:
ordered_cols = [x for x, _ in sorted(idx_to_r2, key=lambda x: x[1], reverse=True)]
print(ordered_cols)

In [ ]:
rmse_collection = []

for i in range(1, 9):
    lim_cols = ordered_cols[:i]
    model = lm.LinearRegression()
    model.fit(X_train[:, lim_cols], y_train)
    y_pred = model.predict(X_test[:, lim_cols])
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    rmse_collection.append(rmse)

data = {
    'Features': df.columns[ordered_cols],
    'RMSE': rmse_collection,
}

pd.DataFrame(data)